In [1]:
import numpy as np
import pandas as pd
import os

# Récupérer le fichier parent en tant que BASE_DIR
BASE_DIR = "C:/Users/mangg/projects/rakutenteam"
os.chdir(BASE_DIR)

In [2]:
df = pd.read_csv("data/clean/X_train.csv", index_col=0)
df.head()

,designation,description,productid,imageid,language,designation_translated,description_translated
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,de,Olivia : Carnet personnalisé / 150 pages / gri...,NaN
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,fr,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,fr,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,en,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN
4,La Guerre Des Tuques,Luc a des idées de grandeur. Il veut organiser...,278535884,1077757786,fr,La Guerre Des Tuques,Luc a des idées de grandeur. Il veut organiser...


In [4]:
df.designation_translated.to_list()

['Olivia : Carnet personnalisé / 150 pages / grille à points / Ca Din A5 / motif roses',
 "Journal Des Arts (Le) N° 133 Du 28/09/2001 - L'art Et Son Marche Salon D'art Asiatique A Paris - Jacques Barrere - Francois Perrier - La Reforme Des Ventes Aux Encheres Publiques - Le Sna Fete Ses Cent Ans.",
 'Grand Stylet Ergonomique Bleu Gamepad Nintendo Wii U - Speedlink Pilot Style',
 'Peluche Donald - Europe - Disneyland 2000 (Marionnette À Doigt)',
 'La Guerre Des Tuques',
 'Afrique Contemporaine N° 212 Hiver 2004 - Dossier Japon / Afrique',
 'Christof E : Sur les traces des processus éducatifs',
 'Conquérant Sept Cahier Couverture Polypro 240 X 320 Mm 96 Pages 90g Seyès Incolore',
 'Puzzle Scooby-Doo Avec Poster 2x35 Pieces',
 'Tente Pliante V3s5-Pro Pvc Blanc - 3 X 4m50 - Longueur : 4m50 Largeur : 3 M Blanc H',
 "Chaise Sxw inspirée d'Eames - Rose - Noir",
 'Fauteuil Chesterfield Brenton 100% Cuir De Buffle - Vert Empire',
 "Royaume paisible, où est l'ours ? Le jeu de blocs à empiler cac

In [ ]:
df["full_text"] = df["designation_translated"] + " " + df["description_translated"].fillna("")
df.full_text.head()
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
vectorizer.fit(df.full_text)
vectorized_text = vectorizer.transform(df.full_text)
vectorized_text.shape
selected_words = vectorizer.get_feature_names_out()
df_word_vecs = pd.DataFrame(vectorized_text.toarray(), columns=selected_words)
new_df = pd.concat([df, df_word_vecs], axis=1)
new_df.shape
new_df.columns
data = new_df.drop(columns=["designation", "description", "full_text", "productid", "imageid", "designation_translated",
                            "description_translated", "language"])
data.head()
target = pd.read_csv("data/clean/Y_train.csv", index_col=0)["prdtypecode"]
target.shape
target.head()
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
clf = SVC()
clf.fit(X_train, y_train.values)
clf.score(X_test, y_test)
import numpy as np

param_grid = {
    "C": [np.logspace(-4, 2, 10)],
    "kernel": ["linear", "poly", "rbf", "sigmoid", "precomputed"],
    "degree": range(6)
}
clf = SVC()

grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train.values)
results = pd.DataFrame(gridsearch.cv_results_)
final_clf = gridsearch.best_estimator_
final_clf.score(X_test, y_test)
from sklearn.metrics import classification_report

y_pred = final_clf.predict(X_test)
print(classification_report(y_test, y_pred))
pd.crosstab(y_test, y_pred, rownames=["Classes Réelles"], colnames=["Prédictions"], margins=True)

In [ ]:
#(19[\d]{2})